In [43]:
import torchvision
import torch
from PIL import Image
import torch.nn as nn
from torch import optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
import os

In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
num_classes = 2
model = torchvision.models.alexnet(pretrained = True)
# replace the last classifier
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

In [46]:
for param in model.parameters():
    param.requires_grad = False

for param in model.classifier[6].parameters():
    param.requires_grad = True

In [47]:
class PNGDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.image_paths = [os.path.join(folder_path, fname) for fname in os.listdir(folder_path) if fname.endswith('.png')]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')  # Ensure images are in RGB format
        if self.transform:
            image = self.transform(image)
        if 'benign' in image_path:
            label = 0
        else:
            label = 1
        return image, label

image_folder_path = '/content/drive/MyDrive/patch_outputs/'


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = PNGDataset(folder_path=image_folder_path, transform=transform)

dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [48]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 5  # Number of epochs
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)


        optimizer.zero_grad()


        outputs = model(images)
        loss = criterion(outputs, labels)


        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(dataloader):.4f}')

print('Training complete!')

Epoch [1/5], Loss: 0.3824
Epoch [2/5], Loss: 0.1221
Epoch [3/5], Loss: 0.0971
Epoch [4/5], Loss: 0.0791
Epoch [5/5], Loss: 0.0692
Training complete!


In [49]:
running_loss

0.4841033435659483